In [6]:
import cv2
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io
import heapq
import xlsxwriter
from xlutils.copy import copy    
from xlrd import open_workbook
import os
import math

In [7]:
# Search all data with _highBW.mat and save to files
path = os.path.abspath("/home/peppermint/Desktop/Patients")
print(path)
files = []
paths=[];
# Find file with _ute.ni 

for r, d, f in os.walk(path):
    d.sort()
    for file in f:
        if '_highBW.mat' in file:
            index=-5;
            name=[];
            find_=False;
            #print(r)
            while(not find_):
                if(file[index]=='_'):
                    find_= True;
                index=index-1;
                if(find_):
                    name.append(file[0:index+1]);
            files.append(name)
            paths.append(r)


/home/peppermint/Desktop/Patients


In [30]:
excel_ro = open_workbook("/home/peppermint/Desktop/first_try.xls")
excel = copy(excel_ro)  # creates a writeable copy
# find the last line
sheet1 = excel.get_sheet(0)  # get a first sheet
tempt_sheet = excel_ro.sheet_by_index(0)
colx = 0
rowx = tempt_sheet.nrows;



for num_file in range(len(files)):
#for num_file in range(5):
    # load data
    
    current_path = paths[num_file]+'/'+str(files[num_file][0])+'_highBW.mat';
    #print(current_path)
    mat = scipy.io.loadmat(current_path)
    bar2gas = mat['bar2gas']
    mask_reg = mat['mask_reg']
    total_mean=[];
    for i in range(128):
        if(sum(sum(mask_reg[:,:,i]))>=2000):
            current_img=bar2gas[:,:,i];

            output=cv2.connectedComponentsWithStats(mask_reg[:,:,i],4)
            
            num_labels = output[0];
            labels_im = output[1];
            stats=output[2];
            centroid=output[3];
            
            if(num_labels<=2):
                continue

            area=stats[:,4];
            # Delete the background label.
            area=area[1:]

            #Choose the label with largest and second largest except backgrounf
            index_label=np.array(heapq.nlargest(2, range(len(area)), key=area.__getitem__))+1;

            # Find which is left or right
            index_1 = index_label[0]
            index_2 = index_label[1]
            if (centroid[index_1,0]<centroid[index_2,0]):
                left_label=index_1;
                right_label=index_2;
            else:
                left_label=index_2;
                right_label=index_1;

            # Instant of Do morphology we create distance image 

            dist_transform = cv2.distanceTransform(mask_reg[:,:,i],cv2.DIST_L2,3)
            [m,n] = current_img.shape
            sum_left_out=0;
            sum_right_out=0;
            sum_left_in=0;
            sum_right_in=0;

            num_left_out=0;
            num_right_out=0;
            num_left_in=0;
            num_right_in=0;

            for i in range(m):
                for j in range(n):

                    if(labels_im[i,j]==left_label ):

                        if((dist_transform[i,j]>0 and dist_transform[i,j]<=4)):
                            sum_left_in = sum_left_in+ current_img[i,j];
                            num_left_in= num_left_in+1;
                        else:
                            sum_left_out = sum_left_out+ current_img[i,j];
                            num_left_out= num_left_out+1;


                    elif (labels_im[i,j]==right_label):

                        if((dist_transform[i,j]>0 and dist_transform[i,j]<=4)):
                            sum_right_in = sum_right_in+ current_img[i,j];
                            num_right_in= num_right_in+1;
                        else:
                            sum_right_out = sum_right_out+ current_img[i,j];
                            num_right_out= num_right_out+1;

            
            mean=[];
            have_nan = False;
            if num_left_in!=0:
                mean.append(sum_left_in/num_left_in);
            else:
                mean.append(np.nan);
                have_nan = True;
            if num_left_out!=0:
                mean.append(sum_left_out/num_left_out);
            else:
                mean.append(np.nan);
                have_nan = True;
            if num_right_in!=0:
                mean.append(sum_right_in/num_right_in);
            else:
                mean.append(np.nan);
                have_nan = True;
            if num_right_out!=0:
                mean.append(sum_right_out/num_right_out);
            else:
                mean.append(np.nan);
                have_nan = True;
            
            #if(not have_nan):
            #    total_mean.append(mean);
            total_mean.append(mean);
            
    total_mean=np.array(total_mean);
    total_mean=np.nanmean(total_mean,axis=0)
    #total_mean=np.mean(total_mean,axis=0)
    
    print(total_mean)
    name = files[num_file][0];
    colx = 0
    try:
        sheet1.write(rowx,colx, name)
        sheet1.write(rowx,colx+1, total_mean[0])
        sheet1.write(rowx,colx+2, total_mean[1])
        sheet1.write(rowx,colx+3, total_mean[2])
        sheet1.write(rowx,colx+4, total_mean[3])
        rowx = rowx+1;
    except:
        print('Something not right')

excel.save("/home/peppermint/Desktop/first_try.xls")


[0.44942568 0.48625125 0.57598128 0.5874381 ]
[0.58477691 0.67371577 0.78556477 0.80980193]
[0.40284972 0.42399931 0.58011768 0.49725368]
[0.53455037 0.54226847 0.61079109 0.64953269]
[0.61771347 0.57210647 0.65079844 0.6675693 ]
[0. 0. 0. 0.]
[1.12914195 1.24954963 1.30586691 1.40652257]
[0.24798466 0.30497217 0.24142502 0.31433662]
[0.20313329 0.28835683 0.15569568 0.24390242]
[0.4894935  0.46967169 0.43365263 0.48710742]
[0.73166569 0.8067421  0.75074205 0.88034357]
[0.88981582 1.12835693 0.83156031 1.1967333 ]
[0.58672972 0.73306102 0.64544075 0.78431062]
[0.6392632  0.71056378 0.69226367 0.75607176]
[0.72538446 0.84978139 0.74459058 0.9277523 ]
[0.46035684 0.42474713 0.43631812 0.43196266]
[0.70968606 0.84318489 0.85214847 0.90341348]
[0.62976084 1.12731481 0.8346945  1.13574523]
nan
Something not right


/home/peppermint/.virtualenvs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:117: RuntimeWarning: Mean of empty slice


[0.78338511 0.76418992 0.90019768 0.90351486]
[0.78898806 0.99098742 0.8068832  1.01838569]
[1.01185012 1.0805645  0.94580961 1.15482652]
[0.54947689 0.60061915 0.5854421  0.61969856]
[1.0025756  1.09702162 1.07045721 1.13922166]
[0.92249365 0.92267966 1.03779489 1.05123738]
nan
Something not right
[0.71360415 0.71312215 0.77625238 0.75557714]
[0.85134328 0.9020659  0.89116106 0.94733551]
[1.3259439  1.47733853 1.61314305 1.78365472]
nan
Something not right
[0.63110282 0.64126696 0.70703924 0.73909747]
[0.9073171  0.98708137 0.93660762 0.93122272]
[0.76819055 0.92080032 0.77358081 0.89166596]
[0.99512794 1.03624394 0.87882617 0.87841955]
[0.63064261 0.63598913 0.62829574 0.64083309]
[0.96488121 0.94731895 0.97906202 0.99647412]
[0.71967521 0.75339842 0.71784281 0.75032215]
[0.77184358 0.79767538 0.86382781 0.92418531]
[0.57488297 0.60212195 0.56851629 0.60837666]
[1.15567207 1.20334764 1.17862261 1.16472122]
[0.69944593 0.70315969 0.76890799 0.76385307]
[0.53800594 0.5635286  0.5392624